<a href="https://colab.research.google.com/github/Sunauraa/RAG-Model-for-obesity-questions/blob/main/RAG_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain-community langchain faiss-cpu CTransformers

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.who.int/news-room/fact-sheets/detail/obesity-and-overweight")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 50,
)

chunks = text_splitter.split_documents(docs)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")

/tmp/ipython-input-2666659659.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warn

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
from langchain_community.llms import CTransformers

llm = CTransformers(
    model = "/content/tinyllama-1.1b-chat-v1.0.Q8_0.gguf",
    model_type = 'llama',
    max_new_tokens = 512,
    temperature = 0.01
)

In [ ]:
llm_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chaintype = "stuff"
    retriever = vectorstore.as_retriever(search_kwargs = {'k':3}),
    return_source_documents = False,
    chain_type_kwargs = {'prompt'}
)

In [ ]:
question = "I'm overweight. How can I reduce fat"
response = llm_chain.invoke({"query":question})